In [ ]:
import tensorflow as tf
import json
from types import SimpleNamespace
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import numpy as np


%run data_preprocess.ipynb
%run helper.ipynb
%run wrapper.ipynb
%run models/vae.ipynb
%run visualize.ipynb
%run data_postprocess.ipynb

In [ ]:
helper = PrepareDataForQuantile()
customStopper = CustomStopper()
visualizer = Visualizer()
post_process = DataPostProcess()
wrapper = Wrapper()
model_evaluation = ModelEvaluation()

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
# load json file
with open("Config/model_config.json") as data_file:
    model_param = json.load(data_file)
    model_param = model_param["vae"]

## Refrigerator


In [ ]:
model_param["training"]["batch_size"] = model_param["training"]["batch_sizes"][0]
model_param["preprocessing"]["width"] = model_param["preprocessing"]["widths"][0]
model_param["preprocessing"]["strides"] = model_param["preprocessing"]["strides_list"][0]
model_param["appliance"] = model_param["appliances"][0]

main_mean = model_param["preprocessing"]["main_mean"]
main_std = model_param["preprocessing"]["main_std"]

app_mean = model_param["preprocessing"]["app_mean"]
app_std = model_param["preprocessing"]["app_std"]

# main_mean = 1482.27
# main_std = 1317.26

# app_mean = 204.14
# app_std = 737.71

In [ ]:
x_train, y_train, x_test, y_test = helper.get_data_for_vae(0, 1, model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"])

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

In [ ]:
epochs = model_param["training"]["epoch"]
batch_size = model_param["training"]["batch_size"]

STEPS_PER_EPOCH = x_train.shape[0]//batch_size

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
                float(model_param["training"]["lr"]), 
                decay_steps=STEPS_PER_EPOCH*model_param["training"]["decay_steps"],
                decay_rate=1,
                staircase=False)

optimizer = tf.keras.optimizers.RMSprop(lr_schedule)

In [ ]:
batch_size

In [ ]:
model = create_model(model_param["model"], model_param["config"], model_param["preprocessing"]["width"], optimizer=optimizer)

In [ ]:
list_callbacks = []
patience = model_param["training"]["patience"]
start_epoch = model_param["training"]["start_stopping"]

print("Patience : {}, Start at : {}".format(patience, start_epoch))

es_callback = CustomStopper(monitor='val_loss', patience=patience, start_epoch=start_epoch, mode="auto")

list_callbacks.append(es_callback)

eps_train = np.random.normal(size=(x_train.shape[0], 16))

In [ ]:

tf.config.experimental.reset_memory_stats('GPU:0')

history = model.fit((x_train-main_mean)/main_std, (y_train-app_mean)/app_std, validation_split=model_param["training"]["validation_split"], shuffle=True, 
                                epochs=epochs, batch_size=batch_size, callbacks=list_callbacks, verbose=1, initial_epoch=0)

In [ ]:
mem_info = tf.config.experimental.get_memory_info('GPU:0')
print("🔁 TF training memory (peak):", mem_info['peak'] / (1024 ** 2), "MB")

In [ ]:
x_test.shape

In [ ]:
#test the model
import psutil
import os
process = psutil.Process(os.getpid())
mem_before = process.memory_info().rss
# eps_test = np.random.normal(size=(x_test.shape[0], 16))

y_pred = model.predict(x_test)

mem_after = process.memory_info().rss
print("🧠 TF inference memory (CPU):", (mem_after - mem_before) / (1024 ** 2), "MB")

In [ ]:
import psutil
import os
dummy_input = np.random.normal(size=(2592, model_param["preprocessing"]["width"], 1))
process = psutil.Process(os.getpid())
mem_before = process.memory_info().rss
print("Dummy input shape:", dummy_input.shape)
model.predict(dummy_input)

mem_after = process.memory_info().rss
print("Inference memory used (MB):", (mem_after - mem_before) / (1024 ** 2))


In [ ]:
# y_pred = y_pred.reshape([1, -1])
# y_test = y_test.reshape([1, -1])

In [ ]:
# y_pred.min(), y_pred.max(), y_test.min(), y_test.max()

In [ ]:
y_all_pred = reconstruct(y_pred[:]*app_std+app_mean, model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"], "mean")
x_all = reconstruct(x_test[:], model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"], "mean")
y_all_true = reconstruct(y_test[:], model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"], "mean")

In [ ]:
y_all_pred.shape, x_all.shape, y_all_true.shape

In [ ]:
y_pred[:].shape

In [ ]:
# remove the nan values
y_all_pred = y_all_pred[~np.isnan(y_all_pred)]
y_all_true = y_all_true[~np.isnan(y_all_true)]
x_all = x_all[~np.isnan(x_all)]
# sum of nan values in the array
np.isnan(y_all_pred).sum(), np.isnan(y_all_true).sum(), np.isnan(x_all).sum()


In [ ]:
y_all_pred.shape, x_all.shape, y_all_true.shape

In [ ]:
x_total, y_total_pred, y_total_true = post_process.post_process_vae(x_all, y_all_pred, y_all_true)

In [ ]:
y_total_pred_refrigerator = y_total_pred
y_total_true_refrigerator = y_total_true

In [ ]:
y_total_pred_refrigerator[0, :].shape, y_total_true_refrigerator.shape

In [ ]:
# prepare a dataframe with the true and predicted values
import pandas as pd
output_data = pd.DataFrame({
    'y_total_pred': y_total_pred_refrigerator[0, :],
    'y_total_true': y_total_true_refrigerator[0, :]
})

output_data.to_csv("Results/output/vae_refrigerator_8156_withoutev.csv", index=False)

In [ ]:
data_temp = pd.read_csv("Results/output/vae_refrigerator_661_withoutev.csv")

In [ ]:
y_total_pred2 = data_temp['y_total_pred'].values
y_total_true2 = data_temp['y_total_true'].values

In [ ]:
import matplotlib.pyplot as plt
# plt.figure(figsize=(20,10))
plt.plot(y_total_true[0, :200], label="True")
plt.plot(y_total_pred[0, :200], label="Predicted")
plt.legend()
plt.show()

In [ ]:
mae, eac, nde = wrapper.evaluation_metrics(y_total_true[0, :], y_total_pred[0, :])

print("MAE: ", mae)
print("EAC: ", eac)
print("NDE: ", nde)

In [ ]:
mae, eac, nde = wrapper.evaluation_metrics(y_total_true[0, :], y_total_pred[0, :])

print("MAE: ", mae)
print("EAC: ", eac)
print("NDE: ", nde)

## EV

In [ ]:
# write the same code for clotheswasher1
model_param["training"]["batch_size"] = model_param["training"]["batch_sizes"][1]
model_param["preprocessing"]["width"] = model_param["preprocessing"]["widths"][1]
model_param["preprocessing"]["strides"] = model_param["preprocessing"]["strides_list"][1]
model_param["appliance"] = model_param["appliances"][1]

main_mean = model_param["preprocessing"]["main_mean"]
main_std = model_param["preprocessing"]["main_std"]

app_mean = model_param["preprocessing"]["app_mean"]
app_std = model_param["preprocessing"]["app_std"]

x_train_wm, y_train_wm, x_test_wm, y_test_wm = helper.get_data_for_vae(1, 2, model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"])

In [ ]:

epochs = model_param["training"]["epoch"]
batch_size = model_param["training"]["batch_size"]

STEPS_PER_EPOCH = x_train.shape[0]//batch_size

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
                float(model_param["training"]["lr"]), 
                decay_steps=STEPS_PER_EPOCH*model_param["training"]["decay_steps"],
                decay_rate=1,
                staircase=False)

optimizer = tf.keras.optimizers.RMSprop(lr_schedule)

model_ev = create_model(model_param["model"], model_param["config"], model_param["preprocessing"]["width"], optimizer=optimizer)

In [ ]:

list_callbacks = []
patience = model_param["training"]["patience"]
start_epoch = model_param["training"]["start_stopping"]

print("Patience : {}, Start at : {}".format(patience, start_epoch))

es_callback = CustomStopper(monitor='val_loss', patience=patience, start_epoch=start_epoch, mode="auto")

list_callbacks.append(es_callback)

history_ev = model_ev.fit((x_train_wm-main_mean)/main_std, (y_train_wm-app_mean)/app_std, validation_split=model_param["training"]["validation_split"], shuffle=True,
                                epochs=epochs, batch_size=batch_size, callbacks=list_callbacks, verbose=1, initial_epoch=0)

In [ ]:

#test the model
y_pred_ev = model_ev.predict((x_test_wm-main_mean)/main_std, verbose=1)

In [ ]:

y_pred_wm = y_pred_ev.reshape([1, -1])
y_test_wm = y_test_wm.reshape([1, -1])

In [ ]:

y_all_pred_ev = reconstruct(y_pred_ev[:]*app_std+app_mean, model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"], "mean")
x_all_ev = reconstruct(x_test_wm[:], model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"], "mean")
y_all_true_ev = reconstruct(y_test_wm[:], model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"], "mean")


In [ ]:

# remove the nan values
y_all_pred_ev = y_all_pred_ev[~np.isnan(y_all_pred_ev)]
y_all_true_ev = y_all_true_ev[~np.isnan(y_all_true_ev)]
x_all_ev = x_all_ev[~np.isnan(x_all_ev)]
# sum of nan values in the array
np.isnan(y_all_pred_ev).sum(), np.isnan(y_all_true_ev).sum(), np.isnan(x_all_ev).sum()

In [ ]:
x_total_ev, y_total_pred_ev, y_total_true_ev = post_process.post_process_vae(x_all_ev, y_all_pred_ev, y_all_true_ev)

In [ ]:
# prepare a dataframe with the true and predicted values
import pandas as pd
output_data = pd.DataFrame({
    'y_total_pred': y_total_pred_ev[0, :],
    'y_total_true': y_total_true_ev[0, :]
})

output_data.to_csv("Results/output/vae_ev_8156_withev.csv", index=False)

In [ ]:

plt.plot(y_total_true_ev[0, 100:12000], label="True")
plt.plot(y_total_pred_ev[0, 100:12000], label="Predicted")
plt.legend()
plt.show()

In [ ]:

mae, eac, nde = wrapper.evaluation_metrics(y_total_true_ev[0, :], y_total_pred_ev[0, :])

print("MAE: ", mae)
print("EAC: ", eac)
print("NDE: ", nde)

In [ ]:
# without EV

mae, eac, nde = wrapper.evaluation_metrics(y_total_true_wm[0, :], y_total_pred_wm[0, :])

print("MAE: ", mae)
print("EAC: ", eac)
print("NDE: ", nde)

## dishwasher

In [ ]:
# write the full code for refrigerator1
model_param["training"]["batch_size"] = model_param["training"]["batch_sizes"][2]
model_param["preprocessing"]["width"] = model_param["preprocessing"]["widths"][2]
model_param["preprocessing"]["strides"] = model_param["preprocessing"]["strides_list"][2]
model_param["appliance"] = model_param["appliances"][2]

main_mean = model_param["preprocessing"]["main_mean"]
main_std = model_param["preprocessing"]["main_std"]

app_mean = model_param["preprocessing"]["app_mean"]
app_std = model_param["preprocessing"]["app_std"]

x_train_dish, y_train_dish, x_test_dish, y_test_dish = helper.get_data_for_vae(1, 2, model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"])

In [ ]:
x_train_dish.shape, y_train_dish.shape, x_test_dish.shape, y_test_dish.shape

In [ ]:

epochs = model_param["training"]["epoch"]
batch_size = model_param["training"]["batch_size"]

STEPS_PER_EPOCH = x_train_dish.shape[0]//batch_size

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
                float(model_param["training"]["lr"]), 
                decay_steps=STEPS_PER_EPOCH*model_param["training"]["decay_steps"],
                decay_rate=1,
                staircase=False)

optimizer = tf.keras.optimizers.RMSprop(lr_schedule)

model_dish = create_model(model_param["model"], model_param["config"], model_param["preprocessing"]["width"], optimizer=optimizer)

In [ ]:

list_callbacks = []
patience = model_param["training"]["patience"]
start_epoch = model_param["training"]["start_stopping"]

print("Patience : {}, Start at : {}".format(patience, start_epoch))

es_callback = CustomStopper(monitor='val_loss', patience=patience, start_epoch=start_epoch, mode="auto")

list_callbacks.append(es_callback)

history_ref = model_dish.fit((x_train_dish-main_mean)/main_std, (y_train_dish-app_mean)/app_std, validation_split=model_param["training"]["validation_split"], shuffle=True,
                                epochs=epochs, batch_size=batch_size, callbacks=list_callbacks, verbose=1, initial_epoch=0)

In [ ]:
#test the model
y_pred_dish = model_dish.predict((x_test_dish-main_mean)/main_std, verbose=1)

In [ ]:
# y_pred_ref = y_pred_ref.reshape([1, -1])
# y_test_ref = y_test_ref.reshape([1, -1])

In [ ]:

y_all_pred_dish = reconstruct(y_pred_dish[:]*app_std+app_mean, model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"], "mean")
x_all_dish = reconstruct(x_test_dish[:], model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"], "mean")
y_all_true_dish = reconstruct(y_test_dish[:], model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"], "mean")

In [ ]:

# remove the nan values
y_all_pred_dish = y_all_pred_dish[~np.isnan(y_all_pred_dish)]
y_all_true_dish = y_all_true_dish[~np.isnan(y_all_true_dish)]
x_all_dish = x_all_dish[~np.isnan(x_all_dish)]
# sum of nan values in the array
np.isnan(y_all_pred_dish).sum(), np.isnan(y_all_true_dish).sum(), np.isnan(x_all_dish).sum()

In [ ]:

x_total_dish, y_total_pred_dish, y_total_true_dish = post_process.post_process_vae(x_all_dish, y_all_pred_dish, y_all_true_dish)

In [ ]:
# prepare a dataframe with the true and predicted values
import pandas as pd
output_data = pd.DataFrame({
    'y_total_pred': y_total_pred_dish[0, :],
    'y_total_true': y_total_true_dish[0, :]
})

output_data.to_csv("Results/output/vae_dish_8156_withoutev.csv", index=False)

In [ ]:


plt.plot(y_total_true_dish[0, :70000], label="True")
plt.plot(y_total_pred_dish[0, :70000], label="Predicted")
plt.legend()
plt.show()

In [ ]:

mae, eac, nde = wrapper.evaluation_metrics(y_total_true_dish[0, :], y_total_pred_dish[0, :])

print("MAE: ", mae)
print("EAC: ", eac)
print("NDE: ", nde)

In [ ]:
#without ev

mae, eac, nde = wrapper.evaluation_metrics(y_total_true_ref[0, :], y_total_pred_ref[0, :])

print("MAE: ", mae)
print("EAC: ", eac)
print("NDE: ", nde)

## clotheswasher

In [ ]:
# write the full code for microwave1
model_param["training"]["batch_size"] = model_param["training"]["batch_sizes"][3]
model_param["preprocessing"]["width"] = model_param["preprocessing"]["widths"][3]
model_param["preprocessing"]["strides"] = model_param["preprocessing"]["strides_list"][3]
model_param["appliance"] = model_param["appliances"][3]

main_mean = model_param["preprocessing"]["main_mean"]
main_std = model_param["preprocessing"]["main_std"]

app_mean = model_param["preprocessing"]["app_mean"]
app_std = model_param["preprocessing"]["app_std"]

x_train_cw, y_train_cw, x_test_cw, y_test_cw = helper.get_data_for_vae(2, 3, model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"])

In [ ]:
# set value 0 if power is less than 180
y_train_cw[y_train_cw<180] = 0
y_test_cw[y_test_cw<180] = 0

In [ ]:

epochs = model_param["training"]["epoch"]
batch_size = model_param["training"]["batch_size"]

STEPS_PER_EPOCH = x_train_cw.shape[0]//batch_size

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
                float(model_param["training"]["lr"]), 
                decay_steps=STEPS_PER_EPOCH*model_param["training"]["decay_steps"],
                decay_rate=1,
                staircase=False)

optimizer = tf.keras.optimizers.RMSprop(lr_schedule)

model_cw = create_model(model_param["model"], model_param["config"], model_param["preprocessing"]["width"], optimizer=optimizer)

In [ ]:

list_callbacks = []
patience = model_param["training"]["patience"]
start_epoch = model_param["training"]["start_stopping"]

print("Patience : {}, Start at : {}".format(patience, start_epoch))

es_callback = CustomStopper(monitor='val_loss', patience=patience, start_epoch=start_epoch, mode="auto")

list_callbacks.append(es_callback)

history_mw = model_cw.fit((x_train_cw-main_mean)/main_std, (y_train_cw-app_mean)/app_std, validation_split=model_param["training"]["validation_split"], shuffle=True,
                                epochs=epochs, batch_size=batch_size, callbacks=list_callbacks, verbose=1, initial_epoch=0)

In [ ]:

#test the model
y_pred_cw = model_cw.predict((x_test_cw-main_mean)/main_std, verbose=1)

In [ ]:

y_pred_cw = y_pred_cw.reshape([1, -1])
y_test_cw = y_test_cw.reshape([1, -1])

In [ ]:

y_all_pred_cw = reconstruct(y_pred_cw[:]*app_std+app_mean, model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"], "mean")
x_all_cw = reconstruct(x_test_cw[:], model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"], "mean")
y_all_true_cw = reconstruct(y_test_cw[:], model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"], "mean")

In [ ]:

# remove the nan values
y_all_pred_cw = y_all_pred_cw[~np.isnan(y_all_pred_cw)]
y_all_true_cw = y_all_true_cw[~np.isnan(y_all_true_cw)]
x_all_cw = x_all_cw[~np.isnan(x_all_cw)]
# sum of nan values in the array
np.isnan(y_all_pred_cw).sum(), np.isnan(y_all_true_cw).sum(), np.isnan(x_all_cw).sum()

In [ ]:

x_total_cw, y_total_pred_cw, y_total_true_cw = post_process.post_process_vae(x_all_cw, y_all_pred_cw, y_all_true_cw)

In [ ]:
y_total_pred_cw[y_total_pred_cw<180] = 0

In [ ]:
plt.plot(y_total_true_cw[0, 500:37060], label="True")
plt.plot(y_total_pred_cw[0, 500:37060], label="Predicted")
plt.legend()
plt.show()

In [ ]:
# prepare a dataframe with the true and predicted values
import pandas as pd
output_data = pd.DataFrame({
    'y_total_pred': y_total_pred_cw[0, :],
    'y_total_true': y_total_true_cw[0, :]
})

output_data.to_csv("Results/output/vae_cloth_8156_withoutev.csv", index=False)

In [ ]:

mae, eac, nde = wrapper.evaluation_metrics(y_total_true_cw[0, :], y_total_pred_cw[0, :])

print("MAE: ", mae)
print("EAC: ", eac)
print("NDE: ", nde)


In [ ]:

mae, eac, nde = wrapper.evaluation_metrics(y_total_true_cw[0, :], y_total_pred_cw[0, :])

print("MAE: ", mae)
print("EAC: ", eac)
print("NDE: ", nde)


In [ ]:
#without ev

mae, eac, nde = wrapper.evaluation_metrics(y_total_true_mw[0, :], y_total_pred_mw[0, :])

print("MAE: ", mae)
print("EAC: ", eac)
print("NDE: ", nde)

## air conditioner

In [ ]:
# write the full code for microwave1
model_param["training"]["batch_size"] = model_param["training"]["batch_sizes"][4]
model_param["preprocessing"]["width"] = model_param["preprocessing"]["widths"][4]
model_param["preprocessing"]["strides"] = model_param["preprocessing"]["strides_list"][4]
model_param["appliance"] = model_param["appliances"][4]

main_mean = model_param["preprocessing"]["main_mean"]
main_std = model_param["preprocessing"]["main_std"]

app_mean = model_param["preprocessing"]["app_mean"]
app_std = model_param["preprocessing"]["app_std"]

x_train_air, y_train_air, x_test_air, y_test_air = helper.get_data_for_vae(3, 4, model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"])

In [ ]:
# set value 0 if power is less than 180
y_train_air[y_train_air<180] = 0
y_test_air[y_test_air<180] = 0

In [ ]:
epochs = model_param["training"]["epoch"]
batch_size = model_param["training"]["batch_size"]

STEPS_PER_EPOCH = x_train_air.shape[0]//batch_size

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
                float(model_param["training"]["lr"]), 
                decay_steps=STEPS_PER_EPOCH*model_param["training"]["decay_steps"],
                decay_rate=1,
                staircase=False)

optimizer = tf.keras.optimizers.RMSprop(lr_schedule)

model_air = create_model(model_param["model"], model_param["config"], model_param["preprocessing"]["width"], optimizer=optimizer)

In [ ]:

list_callbacks = []
patience = model_param["training"]["patience"]
start_epoch = model_param["training"]["start_stopping"]

print("Patience : {}, Start at : {}".format(patience, start_epoch))

es_callback = CustomStopper(monitor='val_loss', patience=patience, start_epoch=start_epoch, mode="auto")

list_callbacks.append(es_callback)

history_air = model_air.fit((x_train_air-main_mean)/main_std, (y_train_air-app_mean)/app_std, validation_split=model_param["training"]["validation_split"], shuffle=True,
                                epochs=epochs, batch_size=batch_size, callbacks=list_callbacks, verbose=1, initial_epoch=0)

In [ ]:

#test the model
y_pred_air = model_air.predict((x_test_air-main_mean)/main_std, verbose=1)

In [ ]:


y_all_pred_air = reconstruct(y_pred_air[:]*app_std+app_mean, model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"], "mean")
x_all_air = reconstruct(x_test_air[:], model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"], "mean")
y_all_true_air = reconstruct(y_test_air[:], model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"], "mean")

In [ ]:

# remove the nan values
y_all_pred_air = y_all_pred_air[~np.isnan(y_all_pred_air)]
y_all_true_air = y_all_true_air[~np.isnan(y_all_true_air)]
x_all_air = x_all_air[~np.isnan(x_all_air)]
# sum of nan values in the array
np.isnan(y_all_pred_air).sum(), np.isnan(y_all_true_air).sum(), np.isnan(x_all_air).sum()

In [ ]:

x_total_air, y_total_pred_air, y_total_true_air = post_process.post_process_vae(x_all_air, y_all_pred_air, y_all_true_air)

In [ ]:
plt.plot(y_total_true_air[0, 500:37060], label="True")
plt.plot(y_total_pred_air[0, 500:37060], label="Predicted")
plt.legend()
plt.show()

In [ ]:
# prepare a dataframe with the true and predicted values
import pandas as pd
output_data = pd.DataFrame({
    'y_total_pred': y_total_pred_air[0, :],
    'y_total_true': y_total_true_air[0, :]
})

output_data.to_csv("Results/output/vae_air_8156_withoutev.csv", index=False)

In [ ]:

mae, eac, nde = wrapper.evaluation_metrics(y_total_true_air[0, :], y_total_pred_air[0, :])

print("MAE: ", mae)
print("EAC: ", eac)
print("NDE: ", nde)


## Microwave

In [ ]:
# write the full code for microwave1
model_param["training"]["batch_size"] = model_param["training"]["batch_sizes"][5]
model_param["preprocessing"]["width"] = model_param["preprocessing"]["widths"][5]
model_param["preprocessing"]["strides"] = model_param["preprocessing"]["strides_list"][5]
model_param["appliance"] = model_param["appliances"][5]

main_mean = model_param["preprocessing"]["main_mean"]
main_std = model_param["preprocessing"]["main_std"]

app_mean = model_param["preprocessing"]["app_mean"]
app_std = model_param["preprocessing"]["app_std"]

x_train_mic, y_train_mic, x_test_mic, y_test_mic = helper.get_data_for_vae(4, 5, model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"])

In [ ]:
epochs = model_param["training"]["epoch"]
batch_size = model_param["training"]["batch_size"]

STEPS_PER_EPOCH = x_train_mic.shape[0]//batch_size

lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
                float(model_param["training"]["lr"]), 
                decay_steps=STEPS_PER_EPOCH*model_param["training"]["decay_steps"],
                decay_rate=1,
                staircase=False)

optimizer = tf.keras.optimizers.RMSprop(lr_schedule)

model_mic = create_model(model_param["model"], model_param["config"], model_param["preprocessing"]["width"], optimizer=optimizer)

In [ ]:

list_callbacks = []
patience = model_param["training"]["patience"]
start_epoch = model_param["training"]["start_stopping"]

print("Patience : {}, Start at : {}".format(patience, start_epoch))

es_callback = CustomStopper(monitor='val_loss', patience=patience, start_epoch=start_epoch, mode="auto")

list_callbacks.append(es_callback)

history_mic = model_mic.fit((x_train_mic-main_mean)/main_std, (y_train_mic-app_mean)/app_std, validation_split=model_param["training"]["validation_split"], shuffle=True,
                                epochs=epochs, batch_size=batch_size, callbacks=list_callbacks, verbose=1, initial_epoch=0)

In [ ]:

#test the model
y_pred_mic = model_mic.predict((x_test_mic-main_mean)/main_std, verbose=1)

In [ ]:


y_all_pred_mic = reconstruct(y_pred_mic[:]*app_std+app_mean, model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"], "mean")
x_all_mic = reconstruct(x_test_mic[:], model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"], "mean")
y_all_true_mic = reconstruct(y_test_mic[:], model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"], "mean")

In [ ]:

# remove the nan values
y_all_pred_mic = y_all_pred_mic[~np.isnan(y_all_pred_mic)]
y_all_true_mic = y_all_true_mic[~np.isnan(y_all_true_mic)]
x_all_mic = x_all_mic[~np.isnan(x_all_mic)]
# sum of nan values in the array
np.isnan(y_all_pred_mic).sum(), np.isnan(y_all_true_mic).sum(), np.isnan(x_all_mic).sum()

In [ ]:

x_total_mic, y_total_pred_mic, y_total_true_mic = post_process.post_process_vae(x_all_mic, y_all_pred_mic, y_all_true_mic)

In [ ]:
plt.plot(y_total_true_mic[0, 500:37060], label="True")
plt.plot(y_total_pred_mic[0, 500:37060], label="Predicted")
plt.legend()
plt.show()

In [ ]:
# prepare a dataframe with the true and predicted values
import pandas as pd
output_data = pd.DataFrame({
    'y_total_pred': y_total_pred_mic[0, :],
    'y_total_true': y_total_true_mic[0, :]
})

output_data.to_csv("Results/output/vae_mic_8156_withoutev.csv", index=False)

In [ ]:

mae, eac, nde = wrapper.evaluation_metrics(y_total_true_mic[0, :], y_total_pred_mic[0, :])

print("MAE: ", mae)
print("EAC: ", eac)
print("NDE: ", nde)


## others(White appliances)

In [ ]:
# write the full code for others
model_param["training"]["batch_size"] = model_param["training"]["batch_sizes"][6]
model_param["preprocessing"]["width"] = model_param["preprocessing"]["widths"][6]
model_param["preprocessing"]["strides"] = model_param["preprocessing"]["strides_list"][6]
model_param["appliance"] = model_param["appliances"][6]
main_mean = model_param["preprocessing"]["main_mean"]
main_std = model_param["preprocessing"]["main_std"]
app_mean = model_param["preprocessing"]["app_mean"]
app_std = model_param["preprocessing"]["app_std"]
x_train_others, y_train_others, x_test_others, y_test_others = helper.get_data_for_vae(5, 6, model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"])


In [ ]:
epochs = model_param["training"]["epoch"]
batch_size = model_param["training"]["batch_size"]
STEPS_PER_EPOCH = x_train_others.shape[0]//batch_size
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
                float(model_param["training"]["lr"]), 
                decay_steps=STEPS_PER_EPOCH*model_param["training"]["decay_steps"],
                decay_rate=1,
                staircase=False)
optimizer = tf.keras.optimizers.RMSprop(lr_schedule)
model_others = create_model(model_param["model"], model_param["config"], model_param["preprocessing"]["width"], optimizer=optimizer)

In [ ]:
#continue
list_callbacks = []
patience = model_param["training"]["patience"]
start_epoch = model_param["training"]["start_stopping"]
print("Patience : {}, Start at : {}".format(patience, start_epoch))
es_callback = CustomStopper(monitor='val_loss', patience=patience, start_epoch=start_epoch, mode="auto")
list_callbacks.append(es_callback)
history_others = model_others.fit((x_train_others-main_mean)/main_std, (y_train_others-app_mean)/app_std, validation_split=model_param["training"]["validation_split"], shuffle=True,
                                epochs=epochs, batch_size=batch_size, callbacks=list_callbacks, verbose=1, initial_epoch=0)

In [ ]:
# continue
y_pred_others = model_others.predict((x_test_others-main_mean)/main_std, verbose=1)

In [ ]:
#continue

y_all_pred_others = reconstruct(y_pred_others[:]*app_std+app_mean, model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"], "mean")
x_all_others = reconstruct(x_test_others[:], model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"], "mean")
y_all_true_others = reconstruct(y_test_others[:], model_param["preprocessing"]["width"], model_param["preprocessing"]["strides"], "mean")

In [ ]:


# remove the nan values
y_all_pred_others = y_all_pred_others[~np.isnan(y_all_pred_others)]
y_all_true_others = y_all_true_others[~np.isnan(y_all_true_others)]
x_all_others = x_all_others[~np.isnan(x_all_others)]
# sum of nan values in the array
np.isnan(y_all_pred_others).sum(), np.isnan(y_all_true_others).sum(), np.isnan(x_all_others).sum()

In [ ]:
x_total_others, y_total_pred_others, y_total_true_others = post_process.post_process_vae(x_all_others, y_all_pred_others, y_all_true_others)

In [ ]:
plt.plot(y_total_true_others[0, 500:37060], label="True")
plt.plot(y_total_pred_others[0, 500:37060], label="Predicted")
plt.legend()
plt.show()

In [ ]:

# prepare a dataframe with the true and predicted values
import pandas as pd
output_data = pd.DataFrame({
    'y_total_pred': y_total_pred_others[0, :],
    'y_total_true': y_total_true_others[0, :]
})
output_data.to_csv("Results/output/vae_others_8156_withoutev.csv", index=False)

In [ ]:
mae, eac, nde = wrapper.evaluation_metrics(y_total_true_others[0, :], y_total_pred_others[0, :])

print("MAE: ", mae)
print("EAC: ", eac)
print("NDE: ", nde)

In [ ]:
mae, eac, nde = wrapper.evaluation_metrics(y_total_true_others[0, :], y_total_pred_others[0, :])

print("MAE: ", mae)
print("EAC: ", eac)
print("NDE: ", nde)